# Introduction to XGBoost with RAPIDS
#### By Paul Hendricks
-------

While the world’s data doubles each year, CPU computing has hit a brick wall with the end of Moore’s law. For the same reasons, scientific computing and deep learning has turned to NVIDIA GPU acceleration, data analytics and machine learning where GPU acceleration is ideal. 

NVIDIA created RAPIDS – an open-source data analytics and machine learning acceleration platform that leverages GPUs to accelerate computations. RAPIDS is based on Python, has pandas-like and Scikit-Learn-like interfaces, is built on Apache Arrow in-memory data format, and can scale from 1 to multi-GPU to multi-nodes. RAPIDS integrates easily into the world’s most popular data science Python-based workflows. RAPIDS accelerates data science end-to-end – from data prep, to machine learning, to deep learning. And through Arrow, Spark users can easily move data into the RAPIDS platform for acceleration.

In this notebook, we'll show the acceleration one can gain by using GPUs with XGBoost in RAPIDS.

**Table of Contents**

* Setup
* Load Libraries
* Load/Simulate Data
  * Load Data
  * Simulate Data
  * Split Data
  * Check Dimensions
* Convert NumPy data to DMatrix format
* Set Parameters
* Train Model
* Conclusion

## Setup

This notebook was tested using the `nvcr.io/nvidia/rapidsai/rapidsai:0.5-cuda10.0-runtime-ubuntu18.04-gcc7-py3.7` Docker container from [NVIDIA GPU Cloud](https://ngc.nvidia.com) and run on the NVIDIA Tesla V100 GPU. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. 

If you think you have found a bug or an error, please file an issue here: https://github.com/rapidsai/notebooks/issues

To start, let's see what hardware we're working with.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # To disable GPUs (https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars)

## Load Libraries

Let's load some of the libraries within the RAPIDs ecosystem and see which versions we have.

In [2]:
import numpy as np; print('numpy Version:', np.__version__)
import pandas as pd; print('pandas Version:', pd.__version__)
import xgboost as xgb; print('XGBoost Version:', xgb.__version__)

numpy Version: 1.18.4
pandas Version: 0.25.3


/opt/conda/envs/rapids/lib/python3.6/site-packages/cudf/utils/gpu_utils.py:120: UserWarning: No NVIDIA GPU detected
  warnings.warn("No NVIDIA GPU detected")


XGBoost Version: 1.0.2


## Load/Simulate data

### Load Data

We can load the data using `pandas.read_csv`.

In [3]:
# helper function for loading data
def load_data(filename, n_rows):
    if n_rows >= 1e9:
        df = pd.read_csv(filename)
    else:
        df = pd.read_csv(filename, nrows=n_rows)
    return df.values.astype(np.float32)

In [4]:
%%time

n_rows = int(1e5)
dataset = load_data('data.csv', n_rows)
print(dataset.shape)

(99999, 101)
CPU times: user 1.97 s, sys: 177 ms, total: 2.15 s
Wall time: 2.15 s


### Split Data

We'll split our dataset into a 80% training dataset and a 20% validation dataset.

In [5]:
# identify shape and indices
n_rows, n_columns = dataset.shape
train_size = 0.80
train_index = int(n_rows * train_size)

# split X, y
X, y = dataset[:, 1:], dataset[:, 0]
del dataset

# split train data
X_train, y_train = X[:train_index, :], y[:train_index]

# split validation data
X_validation, y_validation = X[train_index:, :], y[train_index:]

### Check Dimensions

We can check the dimensions and proportions of our training and validation dataets.

In [6]:
# check dimensions
print('X_train: ', X_train.shape, X_train.dtype, 'y_train: ', y_train.shape, y_train.dtype)
print('X_validation', X_validation.shape, X_validation.dtype, 'y_validation: ', y_validation.shape, y_validation.dtype)

# check the proportions
total = X_train.shape[0] + X_validation.shape[0]
print('X_train proportion:', X_train.shape[0] / total)
print('X_validation proportion:', X_validation.shape[0] / total)

X_train:  (79999, 100) float32 y_train:  (79999,) float32
X_validation (20000, 100) float32 y_validation:  (20000,) float32
X_train proportion: 0.7999979999799998
X_validation proportion: 0.2000020000200002


## Convert NumPy data to DMatrix format

With out data simulated and formatted as NumPy arrays, our next step is to convert this to a `DMatrix` object that XGBoost can work with. We can instantiate an object of the `xgboost.DMatrix` by passing in the feature matrix as the first argument followed by the label vector using the `label=` keyword argument. To learn more about XGBoost's support for data structures other than NumPy arrays, see the documentation for the Data Interface:


https://xgboost.readthedocs.io/en/latest/python/python_intro.html#data-interface


In [7]:
%%time

dtrain = xgb.DMatrix(X_train, label=y_train)
dvalidation = xgb.DMatrix(X_validation, label=y_validation)

CPU times: user 241 ms, sys: 32.6 ms, total: 273 ms
Wall time: 277 ms


## Set Parameters

There are a number of parameters that can be set before XGBoost can be run. 

* General parameters relate to which booster we are using to do boosting, commonly tree or linear model
* Booster parameters depend on which booster you have chosen
* Learning task parameters decide on the learning scenario. For example, regression tasks may use different parameters with ranking tasks.

For more information on the configurable parameters within the XGBoost module, see the documentation here:


https://xgboost.readthedocs.io/en/latest/parameter.html

In [8]:
max_depth = 5
num_rounds = 10
eta = 0.3

In [9]:
# instantiate params
params = {}

# general params
general_params = {'silent': 1, 'max_depth': max_depth, 'eta': eta}
params.update(general_params)

# booster params
n_gpus = 0
booster_params = {}

if n_gpus != 0:
    booster_params['tree_method'] = 'gpu_hist'
    booster_params['n_gpus'] = n_gpus
params.update(booster_params)

# learning task params
learning_task_params = {'eval_metric': 'auc', 'objective': 'binary:logistic'}
params.update(learning_task_params)
print(params)

{'silent': 1, 'max_depth': 5, 'eta': 0.3, 'eval_metric': 'auc', 'objective': 'binary:logistic'}


## Train Model

Now it's time to train our model! We can use the `xgb.train` function and pass in the parameters, training dataset, the number of boosting iterations, and the list of items to be evaluated during training. For more information on the parameters that can be passed into `xgb.train`, check out the documentation:


https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.train

In [10]:
# model training settings
evallist = [(dvalidation, 'validation'), (dtrain, 'train')]
num_round = 10

In [11]:
%%time

bst = xgb.train(params, dtrain, num_rounds, evallist)

[0]	validation-auc:0.50099	train-auc:0.53175
[1]	validation-auc:0.50143	train-auc:0.54148
[2]	validation-auc:0.50219	train-auc:0.54984
[3]	validation-auc:0.50108	train-auc:0.55721
[4]	validation-auc:0.50124	train-auc:0.56398
[5]	validation-auc:0.50142	train-auc:0.56986
[6]	validation-auc:0.50184	train-auc:0.57438
[7]	validation-auc:0.50300	train-auc:0.57830
[8]	validation-auc:0.50217	train-auc:0.58259
[9]	validation-auc:0.50349	train-auc:0.58676
CPU times: user 46.2 s, sys: 3.94 s, total: 50.2 s
Wall time: 691 ms


## Conclusion

To learn more about RAPIDS, be sure to check out: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)